In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import warnings
warnings.filterwarnings("ignore")
import os, sys
os.chdir(r"C:\Users\ACER\Desktop\Terragon recruitment")

In [2]:
train_set = pd.read_csv(r"Train.csv")
test_set = pd.read_csv(r"Test.csv")

In [3]:
train_sample = train_set
train_sample.head()

,customer_id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,RGD002844,31,management,married,tertiary,no,81,yes,no,cellular,29,oct,164,2,-1,0,unknown,1
1,RGD003806,62,retired,married,secondary,no,569,no,no,cellular,3,aug,187,2,180,6,success,1
2,RGD008310,35,technician,married,tertiary,no,432,no,no,cellular,12,aug,104,8,-1,0,unknown,0
3,RGD001840,43,management,married,tertiary,no,1429,yes,no,cellular,7,may,1030,1,169,3,success,1
4,RGD005881,29,blue-collar,married,primary,no,25,yes,no,unknown,4,jun,188,2,-1,0,unknown,0


In [4]:
train_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8929 entries, 0 to 8928
Data columns (total 18 columns):
customer_id    8929 non-null object
age            8929 non-null int64
job            8929 non-null object
marital        8929 non-null object
education      8929 non-null object
default        8929 non-null object
balance        8929 non-null int64
housing        8929 non-null object
loan           8929 non-null object
contact        8929 non-null object
day            8929 non-null int64
month          8929 non-null object
duration       8929 non-null int64
campaign       8929 non-null int64
pdays          8929 non-null int64
previous       8929 non-null int64
poutcome       8929 non-null object
deposit        8929 non-null int64
dtypes: int64(8), object(10)
memory usage: 1.2+ MB


In [5]:
categorical_column = ['job','marital', 'education', 'default','housing', 'loan', 'contact', 'month', 'poutcome']
discard_numeric = 'duration'

In [6]:
check_train = pd.get_dummies(train_sample, columns = categorical_column)

In [7]:
re_valuate = check_train.drop('duration', axis = 1)

In [8]:
re_valuate.describe()

,age,balance,day,campaign,pdays,previous,deposit,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
count,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,...,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000,8929.000000
mean,41.270915,1545.323440,15.646657,2.520887,51.094523,0.835144,0.471833,0.120282,0.175160,0.029455,...,0.107739,0.025087,0.257252,0.084332,0.034718,0.029903,0.108859,0.048606,0.094523,0.748012
std,11.944464,3335.891813,8.390010,2.787669,108.388595,2.319982,0.499234,0.325309,0.380125,0.169086,...,0.310068,0.156398,0.437144,0.277901,0.183076,0.170328,0.311479,0.215054,0.292572,0.434179
min,18.000000,-6847.000000,1.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,122.000000,8.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,39.000000,553.000000,15.000000,2.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,49.000000,1723.000000,21.000000,3.000000,8.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,95.000000,81204.000000,31.000000,63.000000,842.000000,58.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
preprocess_train = re_valuate
first_preprocess_train =preprocess_train.drop(['customer_id', 'deposit'], axis = 1)

In [10]:
first_preprocess_train.head()
print (first_preprocess_train.shape)

(8929, 50)


In [52]:
# applying dimensionality reduction
# choosing the right number of dimensions

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(first_preprocess_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum <= 0.95) + 1
print (d)

1


In [55]:
pca = PCA(n_components = 1)
pca.fit(first_preprocess_train)
pca.explained_variance_ratio_

array([ 0.99892511])

In [56]:
pca = PCA(n_components = 0.95)
x_reduced = pca.fit_transform(first_preprocess_train)
print (x_reduced.shape)

(8929, 1)


In [57]:
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.cross_validation import train_test_split

In [58]:
x = x_reduced
y = re_valuate['deposit']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

In [59]:
scale = MinMaxScaler()
std_scale = StandardScaler()
x_train_std = std_scale.fit_transform(x_train)
x_test_std = std_scale.fit_transform(x_test)

x_train_scale = scale.fit_transform(x_train_std)
x_test_scale = scale.fit_transform(x_test_std)

In [60]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

knn = KNeighborsClassifier(n_neighbors = 10)

knn.fit(x_train_scale, y_train)
y_pred = knn.predict(x_test_scale)
mse = mean_squared_error(y_pred, y_test)
mse
knn.score(x_train_scale, y_train),knn.score(x_test_scale, y_test)


(0.65826683466330671, 0.53303471444568873)

In [61]:
for n in [1,2,3,4,5,6,7,8,9,10]:
    knn =  KNeighborsClassifier(n_neighbors = n)
    knn.fit(x_train_scale, y_train)
    print (knn.score(x_train_scale, y_train), "       ",knn.score(x_test_scale, y_test), "      ", n)

1.0         0.556550951848        1
0.778384432311         0.53751399776        2
0.77446451071         0.552631578947        3
0.717625647487         0.535274356103        4
0.718745625087         0.545352743561        5
0.688926221476         0.542553191489        6
0.688366232675         0.54647256439        7
0.671286574269         0.541993281075        8
0.671426571469         0.538073908175        9
0.658266834663         0.533034714446        10


In [62]:
from sklearn.cross_validation import cross_val_score
score = cross_val_score(knn, x_train_scale, y_train, cv = 10)
print (np.mean(score))

0.565587745588


In [63]:
from sklearn.metrics import accuracy_score
y_pred = knn.predict(x_test_scale)
mse = mean_squared_error(y_pred, y_test)
print (mse ,accuracy_score(y_test, y_pred))

0.466965285554 0.533034714446


In [64]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C = 100)
lr.fit(x_train_scale, y_train)
lr.score(x_train_scale, y_train), lr.score(x_test_scale, y_test)

(0.54500909981800361, 0.55151175811870101)

In [65]:
for C in [0.00001,0.0001,0.001,0.01,0.1,1,1.0,10,100]:
    lr = LogisticRegression(C = C)
    lr.fit(x_train_scale, y_train)
    print (lr.score(x_train_scale, y_train), "     ", lr.score(x_test_scale, y_test), "     ", C)

0.529609407812       0.522396416573       1e-05
0.529609407812       0.522396416573       0.0001
0.529609407812       0.522396416573       0.001
0.529609407812       0.522396416573       0.01
0.528909421812       0.525755879059       0.1
0.539269214616       0.541993281075       1
0.539269214616       0.541993281075       1.0
0.544729105418       0.54647256439       10
0.545009099818       0.551511758119       100


In [66]:
score = cross_val_score(lr, x_train_scale, y_train, cv = 10)
print (np.mean(score))

0.544729192376


In [67]:
y_pred = lr.predict(x_test_scale)
mse = mean_squared_error(y_pred, y_test)
print (mse ,accuracy_score(y_test, y_pred))

0.448488241881 0.551511758119


In [68]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(70, activation = 'relu', input_shape = (7143,50)))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [69]:
from keras import optimizers
model.compile(optimizer = optimizers.RMSprop(lr = 0.001),
             loss = 'mse', metrics = ['accuracy'])

In [70]:
# implementing confusion matrix
# cross val predict performs kfold cross_validation, but instead of
# returning the evaluation scores, it returns the predictions made on each
# test fold.

from sklearn.model_selection import cross_val_predict
y_train_predict = cross_val_predict(lr, x_train_scale, y_train, cv = 10)
y_train_predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [71]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_train_predict)

array([[3458,  325],
       [2927,  433]])

In [72]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train, y_train_predict)

0.5712401055408971

In [73]:
recall_score(y_train, y_train_predict)

0.12886904761904761

In [74]:
from sklearn.metrics import f1_score
f1_score(y_train, y_train_predict)

0.21029626032054394

In [75]:
from sklearn.model_selection import cross_val_predict
confusion_value, precision, recall, f1 = np.array((2,2)), 0, 0, 0
y_train_predict = cross_val_predict(knn, x_train_scale, y_train, cv = 10)
from sklearn.metrics import confusion_matrix
confusion_value = confusion_matrix(y_train, y_train_predict)
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_train, y_train_predict)
recall = recall_score(y_train, y_train_predict)
f1 = f1_score(y_train, y_train_predict)
print (precision, recall, f1)

0.5522995523 0.403869047619 0.466563520715


In [76]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd.fit(x_train_scale, y_train)
sgd.score(x_train_scale, y_train),sgd.score(x_test_scale, y_test)
y_pred = sgd.predict(x_test_scale)
mse = mean_squared_error(y_pred, y_test)
print (mse ,accuracy_score(y_test, y_pred))


0.472004479283 0.527995520717


In [77]:
from sklearn.model_selection import cross_val_predict
confusion_value, precision, recall, f1 = np.array((2,2)), 0, 0, 0
y_train_predict = cross_val_predict(sgd, x_train_scale, y_train, cv = 10)
from sklearn.metrics import confusion_matrix
confusion_value = confusion_matrix(y_train, y_train_predict)
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_train, y_train_predict)
recall = recall_score(y_train, y_train_predict)
f1 = f1_score(y_train, y_train_predict)
print (precision, recall, f1)

0.493424513414 0.279166666667 0.356586200342


In [78]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth = 9)
dtc.fit(x_train_scale, y_train)
dtc.score(x_train_scale, y_train),dtc.score(x_test_scale, y_test)
y_pred = dtc.predict(x_test_scale)
mse = mean_squared_error(y_pred, y_test)
print (mse ,accuracy_score(y_test, y_pred))

0.437849944009 0.562150055991


In [79]:
for i in range(1,11):
    dtc = DecisionTreeClassifier(max_depth = i)
    dtc.fit(x_train_scale, y_train)
    dtc.score(x_train_scale, y_train),dtc.score(x_test_scale, y_test)
    y_pred = dtc.predict(x_test_scale)
    mse = mean_squared_error(y_pred, y_test)
    print (i, mse ,accuracy_score(y_test, y_pred))

1 0.433930571109 0.566069428891
2 0.43673012318 0.56326987682
3 0.43673012318 0.56326987682
4 0.440649496081 0.559350503919
5 0.435610302352 0.564389697648
6 0.436170212766 0.563829787234
7 0.437849944009 0.562150055991
8 0.436170212766 0.563829787234
9 0.437849944009 0.562150055991
10 0.432250839866 0.567749160134


In [80]:
from sklearn.model_selection import cross_val_predict
confusion_value, precision, recall, f1 = np.array((2,2)), 0, 0, 0
y_train_predict = cross_val_predict(dtc, x_train_scale, y_train, cv = 10)
from sklearn.metrics import confusion_matrix
confusion_value = confusion_matrix(y_train, y_train_predict)
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_train, y_train_predict)
recall = recall_score(y_train, y_train_predict)
f1 = f1_score(y_train, y_train_predict)
print (precision, recall, f1)

0.563407930899 0.427083333333 0.485864228881


In [81]:
from sklearn.ensemble import RandomForestClassifier
rand = RandomForestClassifier(max_depth = 10, n_estimators = 60)
rand.fit(x_train_scale, y_train)
rand.score(x_train_scale, y_train),rand.score(x_test_scale, y_test)
y_pred = rand.predict(x_test_scale)
mse = mean_squared_error(y_pred, y_test)
print (mse ,accuracy_score(y_test, y_pred))

0.442889137738 0.557110862262


In [82]:
rand.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_split': 1e-07,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 60,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [83]:
for i in range(1,11):
    for nam in [5, 10, 20, 30, 40, 50, 60, 70,80]:
        rand = RandomForestClassifier(max_depth = i, n_estimators = nam)
        rand.fit(x_train_scale, y_train)
        rand.score(x_train_scale, y_train),rand.score(x_test_scale, y_test)
        y_pred = rand.predict(x_test_scale)
        mse = mean_squared_error(y_pred, y_test)
        print (i, mse ,accuracy_score(y_test, y_pred), nam)

1 0.43673012318 0.56326987682 5
1 0.437290033595 0.562709966405 10
1 0.437290033595 0.562709966405 20
1 0.438969764838 0.561030235162 30
1 0.437290033595 0.562709966405 40
1 0.43673012318 0.56326987682 50
1 0.437290033595 0.562709966405 60
1 0.437290033595 0.562709966405 70
1 0.437290033595 0.562709966405 80
2 0.437290033595 0.562709966405 5
2 0.43673012318 0.56326987682 10
2 0.436170212766 0.563829787234 20
2 0.435610302352 0.564389697648 30
2 0.436170212766 0.563829787234 40
2 0.43673012318 0.56326987682 50
2 0.435610302352 0.564389697648 60
2 0.435610302352 0.564389697648 70
2 0.434490481523 0.565509518477 80
3 0.438969764838 0.561030235162 5
3 0.441209406495 0.558790593505 10
3 0.438409854423 0.561590145577 20
3 0.438409854423 0.561590145577 30
3 0.438409854423 0.561590145577 40
3 0.437849944009 0.562150055991 50
3 0.438969764838 0.561030235162 60
3 0.439529675252 0.560470324748 70
3 0.441769316909 0.558230683091 80
4 0.440089585666 0.559910414334 5
4 0.440649496081 0.559350503919 

In [84]:
from sklearn.model_selection import cross_val_predict
confusion_value, precision, recall, f1 = np.array((2,2)), 0, 0, 0
y_train_predict = cross_val_predict(rand, x_train_scale, y_train, cv = 10)
from sklearn.metrics import confusion_matrix
confusion_value = confusion_matrix(y_train, y_train_predict)
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_train, y_train_predict)
recall = recall_score(y_train, y_train_predict)
f1 = f1_score(y_train, y_train_predict)
print (precision, recall, f1)

0.555555555556 0.482142857143 0.516252390057


In [85]:
from sklearn.metrics import precision_recall_curve
precisions, recalls, threshold = precision_recall_curve(y_train, y_train_predict)

In [86]:
precision_score(y_train, y_train_predict)

0.55555555555555558

In [87]:
recall_score(y_train, y_train_predict)

0.48214285714285715

In [89]:
from sklearn.ensemble import GradientBoostingClassifier
val_i, val_j, val_k, val_mse = [],[],[],[]
for i in range(1,11):
    for j in [0.001,0.01,0.1,1, 10,100,1000]:
        for k in [5,10,20,30,40,50,60,70,80,90,100]:
            gbrt = GradientBoostingClassifier(random_state = 1, max_depth = i, learning_rate = j, n_estimators = k)
            gbrt.fit(x_train_scale, y_train)
            gbrt.score(x_train_scale, y_train),gbrt.score(x_test_scale, y_test)
            y_pred = gbrt.predict(x_test_scale)
            mse = mean_squared_error(y_pred, y_test)
            val_i.append(i)
            val_j.append(j)
            val_k.append(k)
            val_mse.append(mse)        

In [90]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train, y_train_predict)



In [91]:
def plot_roc_curve(fpr, tpr, label = None):
    plt.plot(fpr, tpr, linewidth = 2, label = label)
    plt.plot([0,1], [0,1], 'k--')
    plt.axis([0,1,0,1])
    plt.xlabel('False positive Rate')
    plt.ylabel('True positive Rate')
    
plot_roc_curve(fpr, tpr)
plt.grid()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, y_train_predict)